# melody basic rnn
## Douglas Eck
This notebook shows you how to generate new performed compositions from a trained Performance RNN model. You'll see how to download a bundle containing a pre-trained model, instantiate and initialize the model and generate new polyphonic performances. The notebook also shows some hyperparameters useful for controlling generation, such as ``temperature``.

To hear the performances it is useful to have fluidsynth and pyfluidsynth installed. One way to install fluidsynth on OS X is via homebrew: ``brew install fluidsynth``.  One way to install pyfluidsynth is via pip: ``pip install pyfluidsynth``.

In [ ]:
import os
from magenta.models.melody_rnn import melody_rnn_sequence_generator
from magenta.protobuf import generator_pb2
from magenta.protobuf import music_pb2

import magenta.music as mm

# Constants.
BUNDLE_DIR = '/home/jovyan/models/'
MODEL_NAME = 'lookback_rnn' # one of 'basic_rnn', 'attention_rnn', 'lookback_rnn'
BUNDLE_NAME = 'lookback_rnn.mag'

In [ ]:
mm.notebook_utils.download_bundle(BUNDLE_NAME, BUNDLE_DIR)
bundle = mm.sequence_generator_bundle.read_bundle_file(os.path.join(BUNDLE_DIR, BUNDLE_NAME))

In [ ]:
generator_map = melody_rnn_sequence_generator.get_generator_map()
generator = generator_map[MODEL_NAME](checkpoint=None, bundle=bundle)

In [ ]:
generator.initialize()

In [ ]:
generator_options = generator_pb2.GeneratorOptions()
generator_options.args['temperature'].float_value = .9  # Higher is more random; 1.0 is default. 
generate_section = generator_options.generate_sections.add(start_time=0, end_time=30)
sequence = generator.generate(music_pb2.NoteSequence(), generator_options)

# Play and view this masterpiece.
mm.plot_sequence(sequence)
mm.play_sequence(sequence, mm.midi_synth.fluidsynth)

In [ ]:
qpm = 160

generator.initialize()
generator_options = generator_pb2.GeneratorOptions()
generator_options.args['temperature'].float_value = .9  # Higher is more random; 1.0 is default. 
generator_options.args['qpm'].float_value = qpm

seconds_per_step = 120.0 / qpm / generator.steps_per_quarter
total_seconds = 64 * seconds_per_step
last_end_time = 0

generate_section = generator_options.generate_sections.add(start_time=last_end_time + seconds_per_step,
    end_time=total_seconds)
sequence = generator.generate(music_pb2.NoteSequence(), generator_options)

In [ ]:
generator_options.args['temperature'].float_value = 1.1  # Higher is more random; 1.0 is default. 
#sequence = generator.generate(music_pb2.NoteSequence(), generator_options)
#mm.plot_sequence(sequence)
#generate_section = generator_options.generate_sections.add(start_time=0, end_time=30)
mm.play_sequence(sequence, mm.midi_synth.fluidsynth)

In [ ]:
from magenta.music.midi_io import sequence_proto_to_pretty_midi, sequence_proto_to_midi_file, midi_file_to_sequence_proto

In [ ]:
sequences = []
for i in range(16):
    temperature = [.6,.8,.9,1.8,.6,.8,.9,1.8][i % 8]
    generator_options.args['temperature'].float_value = temperature
    sequence = generator.generate(music_pb2.NoteSequence(), generator_options)
    sequences.append(sequence)
    pm = sequence_proto_to_pretty_midi(sequence)

In [ ]:
from magenta.music.sequences_lib import concatenate_sequences, quantize_note_sequence

In [ ]:
big_seq = concatenate_sequences(sequences)

In [ ]:
mm.play_sequence(big_seq, mm.midi_synth.fluidsynth)

In [ ]:
mm.play_sequence(quantized_melody, mm.midi_synth.fluidsynth)

In [ ]:
sequence_proto_to_midi_file(big_seq, 'lookback_rnn.mid')

In [ ]:
with open('lookback_rnn.sequence','wb') as f:
    f.write(big_seq.SerializeToString())

In [ ]:
from magenta.music.chord_inference import infer_chords_for_sequence

In [ ]:
root_filename = './dance_4'
root_filename = './dance_5'
root_filename = './funk_18'
root_filename = './funk_19'
root_filename = './funk_24'
melody_filename = root_filename + '_melody.mid'
chords_filename = root_filename + '_chords.mid'
bass_filename = root_filename + '_bass.mid'
melody = midi_file_to_sequence_proto(melody_filename)

In [ ]:
mm.play_sequence(melody, mm.midi_synth.fluidsynth)

In [ ]:
quantized_melody = quantize_note_sequence(melody, 8)
infer_chords_for_sequence(quantized_melody)

In [ ]:
from magenta.music.chords_lib import extract_chords
from magenta.music.chord_symbols_lib import chord_symbol_bass, chord_symbol_pitches, chord_symbol_root
from magenta.music.chords_encoder_decoder import encoder_decoder
from magenta.music.chords_lib import BasicChordRenderer

In [ ]:
#(chord_progression,), d = extract_chords(quantized_melody)

In [ ]:
#chord_seq = chord_progression.to_sequence()

In [ ]:
#[(chord,chord_symbol_pitches(chord)) for chord in chord_progression] 

In [ ]:
#chords = extract_chords(quantized_melody)[0][0].to_sequence()

In [ ]:
#chords

In [ ]:
chord_renderer = BasicChordRenderer(program=63)
chord_renderer.render(quantized_melody)

In [ ]:
import magenta
chords = magenta.protobuf.music_pb2.NoteSequence()
chords.CopyFrom(quantized_melody)
bass = magenta.protobuf.music_pb2.NoteSequence()
bass.CopyFrom(quantized_melody)
notes_to_delete_from_chords = []
notes_to_delete_from_bass = []
idx = 0
for n in quantized_melody.notes:
    if n.program!=63:
        notes_to_delete_from_chords.append(n)
        notes_to_delete_from_bass.append(n)
    else:
        if n.pitch <= 47:
            notes_to_delete_from_chords.append(n)
        else:
            notes_to_delete_from_bass.append(n)
for n in notes_to_delete_from_chords:
    chords.notes.remove(n)
for n in notes_to_delete_from_bass:
    bass.notes.remove(n)
for n in bass.notes:
    n.program = 37


In [ ]:
sequence_proto_to_midi_file(chords, chords_filename)
sequence_proto_to_midi_file(bass, bass_filename)

In [ ]:
#bass_notes = magenta.protobuf.music_pb2.NoteSequence()
#bass_notes.CopyFrom(chords)
#notes_to_delete = []
#for idx, n in enumerate(s.notes):
#    if idx % 4 == 0:
#        bass_notes.notes.remove(n)#notes_to_delete.append(n)
#for n in notes_to_delete:
#    bass_notes.notes.remove(n)

In [ ]:
#mm.play_sequence(s, mm.midi_synth.fluidsynth)

In [ ]:
mm.play_sequence(bass, mm.midi_synth.fluidsynth)

In [ ]:
#sequence_proto_to_midi_file(quantized_melody, 'output_with_chords.mid')

# Experimental stuff

In [ ]:
#from magenta.music.sequences_lib import split_note_sequence

In [ ]:
sequences = split_note_sequence(melody, 30)

In [ ]:
for sequence in sequences:
    mm.play_sequence(sequence, mm.midi_synth.fluidsynth)

In [ ]:
input_sequence = sequences[0]

In [ ]:
def adjust_sequence_times(sequence, delta_time):
    """Adjusts note and total NoteSequence times by `delta_time`."""
    retimed_sequence = music_pb2.NoteSequence()
    retimed_sequence.CopyFrom(sequence)

    for note in retimed_sequence.notes:
    note.start_time += delta_time
    note.end_time += delta_time
    retimed_sequence.total_time += delta_time
    return retimed_sequence

In [ ]:
generator_options = generator_pb2.GeneratorOptions()
generator_options.input_sections.add(
    start_time=0,
    end_time=30)
generator_options.generate_sections.add(
    start_time=30,
    end_time=60)

# Get current temperature setting.
generator_options.args['temperature'].float_value = 0.6

# Generate response.
response_sequence = generator.generate(input_sequence, generator_options)
response_sequence = magenta.music.trim_note_sequence(
    adjust_sequence_times(response_sequence, -30), 0, 30)

In [ ]:
#from magenta.music
import magenta

In [ ]:
#mm.play_sequence(response_sequence, mm.midi_synth.fluidsynth)

In [ ]:
response_sequence.MergeFrom(input_sequence)

In [ ]:
mm.play_sequence(response_sequence, mm.midi_synth.fluidsynth)